<a href="https://colab.research.google.com/github/devika1402/learning-pytorch/blob/main/Bag_of_Words_Text_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bag of Words Text Classifier

The code below implements a simple bag of words text classifier.
- We tokenize the text, create a vocabulary and encode each piece of text in the dataset
- The lookup allows for extracting embeddings for each tokenized inputs
- The embedding vectors are added together with a bias vector
- The resulting vector is referred to as the scores
- The score are applied a softmax to generate probabilities which are used for the classification task

The code used in this notebook was inspired by code from the [official repo](https://github.com/neubig/nn4nlp-code) used in the [CMU Neural Networks for NLP class](http://www.phontron.com/class/nn4nlp2021/schedule.html) by [Graham Neubig](http://www.phontron.com/index.php). 

![img txt](https://github.com/dair-ai/ML-Notebooks/blob/main/img/bow.png?raw=true)


In [1]:
import torch
import random
import torch.nn as nn

### Download the Data

In [2]:
# Capturing the stdout/stderr of a cell:
%%capture 

# download the files
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/dev.txt
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/test.txt
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/train.txt

# create the data folders
!mkdir data data/classes
!cp dev.txt data/classes
!cp test.txt data/classes
!cp train.txt data/classes

### Read the Data

In [3]:
# function to read in data, process each line and split columns by " ||| "
def read_data(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.lower().strip()
            line = line.split(' ||| ')
            data.append(line)
    return data

train_data = read_data('data/classes/train.txt')
test_data = read_data('data/classes/test.txt')

### Contruct the Vocab and Datasets

In [4]:
# creating the word and tag indices
word_to_index = {}
word_to_index["<unk>"] = len(word_to_index) # adds <UNK> to dictionary
tag_to_index = {}

# create word to index dictionary and tag to index dictionary from data
def create_dict(data, check_unk=False):
    for line in data:
        for word in line[1].split(" "):
            if check_unk == False:
                if word not in word_to_index:
                    word_to_index[word] = len(word_to_index)
            else:
                if word not in word_to_index:
                    word_to_index[word] = word_to_index["<unk>"] #unknown word

        if line[0] not in tag_to_index:
            tag_to_index[line[0]] = len(tag_to_index)

create_dict(train_data)
create_dict(test_data, check_unk=True)

# create word and tag tensors from data
def create_tensor(data):
    for line in data:
        yield([word_to_index[word] for word in line[1].split(" ")], tag_to_index[line[0]])

train_data = list(create_tensor(train_data))
test_data = list(create_tensor(test_data))

number_of_words = len(word_to_index)
number_of_tags = len(tag_to_index)

### Model

In [5]:
# cpu or gpu
device = "cuda" if torch.cuda.is_available() else "cpu"

# create a simple neural network with embedding layer, bias, and xavier initialization
class BoW(torch.nn.Module):
    def __init__(self, nwords, ntags):
        super(BoW, self).__init__()
        self.embedding = nn.Embedding(nwords, ntags)
        nn.init.xavier_uniform_(self.embedding.weight) 
        #The goal of Xavier Initialization is to initialize the weights such that the variance of the activations are the same across every layer.

        type = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
        self.bias = torch.zeros(ntags, requires_grad=True).type(type)

    def forward(self, x):   #feedforward function
        emb = self.embedding(x) # seq_len x ntags (for each seq) 
        out = torch.sum(emb, dim=0) + self.bias # ntags
        out = out.view(1, -1) # reshape to (1, ntags)
        return out

### Pretest the Model

In [6]:
# function to convert sentence into tensor using word_to_index dictionary
def sentence_to_tensor(sentence):
    return torch.LongTensor([word_to_index[word] for word in sentence.split(" ")])
    #LongTensor used for 64-bit integer (signed)

# test the sentence_to_tensor function
type = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
out = sentence_to_tensor("i love dogs").type(type)
test_model = BoW(number_of_words, number_of_tags).to(device)
test_model(out)

tensor([[-0.0252, -0.0114, -0.0327,  0.0138,  0.0404]], device='cuda:0',
       grad_fn=<ViewBackward0>)

Output: Tensor values converted from sentences

### Train the Model

In [7]:
# train and test the BoW model
model = BoW(number_of_words, number_of_tags).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
type = torch.LongTensor

# Cross-entropy loss is used when adjusting model weights during training. The aim is to minimize the loss, i.e, the smaller the loss the better the model. A perfect model has a cross-entropy loss of 0.
# The results of the Adam optimizer are generally better than every other optimization algorithms, have faster computation time, and require fewer parameters for tuning. 

if torch.cuda.is_available():
    model.to(device)
    type = torch.cuda.LongTensor

# perform training of the Bow model
def train_bow(model, optimizer, criterion, train_data):
    for ITER in range(10):
        # perform training
        model.train()
        random.shuffle(train_data)
        total_loss = 0.0
        train_correct = 0
        for sentence, tag in train_data:
            sentence = torch.tensor(sentence).type(type)
            tag = torch.tensor([tag]).type(type)
            output = model(sentence)
            predicted = torch.argmax(output.data.detach()).item() #Returns the indices of the maximum value of all elements in the input tensor
            
            loss = criterion(output, tag)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if predicted == tag: train_correct+=1

        # perform testing of the model
        model.eval()
        test_correct = 0
        for sentence, tag in test_data:
            sentence = torch.tensor(sentence).type(type)
            output = model(sentence)
            predicted = torch.argmax(output.data.detach()).item()
            if predicted == tag: test_correct += 1
        
        # print model performance results
        log = f'ITER: {ITER+1} | ' \
            f'train loss/sent: {total_loss/len(train_data):.4f} | ' \
            f'train accuracy: {train_correct/len(train_data):.4f} | ' \
            f'test accuracy: {test_correct/len(test_data):.4f}'
        print(log)

# call the train_bow function
train_bow(model, optimizer, criterion, train_data)

ITER: 1 | train loss/sent: 1.4732 | train accuracy: 0.3701 | test accuracy: 0.4032
ITER: 2 | train loss/sent: 1.1208 | train accuracy: 0.6045 | test accuracy: 0.4032
ITER: 3 | train loss/sent: 0.9124 | train accuracy: 0.7156 | test accuracy: 0.4172
ITER: 4 | train loss/sent: 0.7692 | train accuracy: 0.7654 | test accuracy: 0.4172
ITER: 5 | train loss/sent: 0.6638 | train accuracy: 0.8069 | test accuracy: 0.4136
ITER: 6 | train loss/sent: 0.5822 | train accuracy: 0.8305 | test accuracy: 0.4041
ITER: 7 | train loss/sent: 0.5170 | train accuracy: 0.8543 | test accuracy: 0.4027
ITER: 8 | train loss/sent: 0.4635 | train accuracy: 0.8702 | test accuracy: 0.4054
ITER: 9 | train loss/sent: 0.4190 | train accuracy: 0.8851 | test accuracy: 0.4032
ITER: 10 | train loss/sent: 0.3814 | train accuracy: 0.8935 | test accuracy: 0.4000
